## ResNet + CIFAR10

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8,8)

model = ['KNN','LC','FCN','CNN',"CNN+BN","Resnet"]
accuracy = [0.35,0.39,0.52,0.7,0.78,0.82,]
plt.bar(model,accuracy)
plt.title('CIFAR10 accuracy')
plt.xlabel('model')
plt.ylabel('accuracy')
plt.show()

Как получить 90%+ точности для Resnet
...


Augmentation  + manual decrease LR 

#Работа с реальными данными

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/real_data.png" width="700">


Проблемы:

- нехватка данных
- недостаток размеченных данных
- ошибки в разметке
- дисбаланс датасета


### ImageFolder

Создадим датасет из своих данных для этого достаточно разложить изображения по папкам и использовать класс ImgeFolder

In [ ]:
!wget  http://fmb.images.gan4x4.ru/hse/bt_dataset3.zip
!unzip -oq bt_dataset3.zip

In [ ]:
!ls bike/bike_type/train
!ls bike/bike_type/val

In [ ]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt

train_dataset = ImageFolder("/content/bike/bike_type/train")
val_dataset = ImageFolder("/content/bike/bike_type/val")

fig=plt.figure(figsize=(24, 6))
for i in range(1, 2*7 +1):
    img = val_dataset[i][0]
    fig.add_subplot(2, 7, i)
    plt.imshow(np.asarray(img))
plt.show()


Обратите внимание: на многих кадрах один и тот же велосипед. Хорошо ли это? Что будет если часть кадров попадет в обучающую, а часть в теренировочную выборку.

##Дисбаланс классов

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/unbalance.png" width="400">

In [ ]:
print("Classes: ",val_dataset.classes)
print("Sizes train:",len(train_dataset), 'val', len(val_dataset))

У imageFolder есть свойство classes которое запослняется в соответствии с названиями папок.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def class_hist(dataset):
  plt.figure() 
  unique, counts = np.unique(dataset.targets, return_counts=True)
  ax = plt.bar(unique, counts)
  plt.title('Train objects')
  plt.xticks(unique, dataset.classes)
  plt.show()
  return counts

class_hist(train_dataset)
 

Датасет не сбалансирован. Что делать?

#### 1. Использовать адекватные метрики:
- F1_score
- PR_Curve
- Confusion matrix



```
# targets and preds must be calculated during training

from sklearn.metrics import confusion_matrix
import pandas as pd

conf_matrix = pd.DataFrame(confusion_matrix(targets, preds))

conf_matrix.columns = dataset_val.classes
conf_matrix.index = dataset_val.classes

conf_matrix = conf_matrix.rename_axis('Real')
conf_matrix = conf_matrix.rename_axis('Predicted', axis='columns')

conf_matrix
```

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/conf_matrix.png" width="400">



### 2. Использовать веса в Loss - функции

https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html?highlight=crossentropy#torch.nn.CrossEntropyLoss

Как следует из описания к CrossEntropyLoss можно добавить веса классов. Сравним loss для с весам и без для оних и тех же данных.

In [ ]:
import torch
# Without weights
scores = torch.tensor([[2.,30.],[2.,30.]]) # Scores for two samples batch
target = torch.tensor([0,1]) # First sample belongs to class 0 second to 1 and firse was misclassified
weights = torch.tensor([1,1],dtype = torch.float32)
criterion = torch.nn.CrossEntropyLoss( weight = weights)  
criterion(scores,target)


Добавим к первому классу вес 10. Это условно соответствует ситуации с BMX велосипедами в нашем датасете: их примерно в 10 раз больше чем MTB

In [ ]:
weights = torch.tensor([10,1],dtype = torch.float32)
criterion = torch.nn.CrossEntropyLoss( weight = weights)  
criterion(scores,target)


Лосс вырос так как класс на котором возникла ошибка оказался редким.

Рассчет весов

In [ ]:
#numpy
import numpy as np
_, counts = np.unique(train_dataset.targets, return_counts=True)
weights= np.max(counts) / counts
weights = torch.FloatTensor(weights)
print('Веса классов: ', weights)

In [ ]:
#torch
# https://pytorch.org/docs/stable/generated/torch.unique.html
_, counts = torch.unique(torch.tensor(train_dataset.targets),return_counts = True)
print(counts.max())
weights = counts.max() / counts
print('Классы: ',train_dataset.classes)
print('Веса классов: ', weights)

In [ ]:
# Sklearn
#https://scikit-learn.org/stable/modules/generated/sklearn.utils.class_weight.compute_class_weight.html
from sklearn.utils import class_weight
weights = class_weight.compute_class_weight('balanced',np.unique(train_dataset.targets),train_dataset.targets)
print(weights)
weights / min(weights) # The same as before


### 3. Выборка с повышением или с понижением 
(Up-sample or Down-sample): одно из решений проблемы - сбалансировать данные.

  

```
  Это может быть сделано либо за счет увеличения частоты класса меньшинства, либо за счет уменьшения частоты класса большинства с помощью методов случайной или кластерной выборки.

    Выбор между избыточной или недостаточной выборкой и случайным или кластеризованным определяется бизнес-контекстом и размером данных.

    Обычно `upsampling` предпочтителен, когда общий размер данных небольшой, а понижающая дискретизация полезна, когда у нас есть большой объем данных. Точно так же случайная или кластерная выборка определяется тем, насколько хорошо распределены данные.
```
<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_34.png" width="600">

Фактически мы либо удаляем часть объектов одного из классов либо копируем.

## Недостаток данных

Теперь посмотрим на распределение данных по класам в валидационном датасете

In [ ]:
class_hist(val_dataset)


На 3-х изображениях мы не сможем оценить точность. Давайте на них посмотрим

Очевидно что выкидывать тут уже нечего, и 100 копий одного велосипеда не решат проблемму. В данном случае можно переместить несколько велосипедов из тренировочной выборки в валидационную но поблему дефицита BMX-ов в целом это не решит. 

#### Генерация синтетических данных:

 хотя `upsampling` или `downsampling` помогает сбалансировать данные, дублирование данных увеличивает вероятность переобучения.

Другой подход к решению этой проблемы - создание синтетических данных с помощью данных о классе меньшинств.

Для табличных данных можно использовать методы 

[Synthetic Minority Over-sampling Technique (SMOTE)](https://rikunert.com/SMOTE_explained) или Modified- SMOTE - два таких метода, которые генерируют синтетические данные.

Проще говоря, SMOTE берет точки данных класса меньшинства и создает новые точки данных, которые лежат между любыми двумя ближайшими точками данных, соединенными прямой линией.

Для этого алгоритм вычисляет расстояние между двумя точками данных в пространстве признаков, умножает расстояние на случайное число от 0 до 1 и помещает новую точку данных на этом новом расстоянии от одной из точек данных, используемых для определения расстояния.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_35.png" width="700">

В ряде случаев можно генерировать и изображения.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/synthesis.png" width="900">

## Аугментации


Другой способ получения дополнительных данных.
Применяются для изображений и легко позволяют получить дополнительные данные. 

Сам термин пришел из музыки:

Аугмента́ция (позднелат. augmentatio — увеличение, расширение) — техника ритмической композиции в старинной музыке.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_12.png" width="700">

Собственно механизм трансформаций torcvision который мы использовали с первого занятия, в основном предназначен для аугментации данных.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_13.png" width="1000">


Functional-transforms

https://pytorch.org/vision/stable/transforms.html#functional-transforms

In [ ]:
import torchvision.transforms.functional as TF
import matplotlib.pyplot as plt
from PIL import Image

pil = Image.open('bike/bike_type/train/road/road_1150_5fccd8116e87e.jpeg')
transformed_pil = TF.vflip(pil)
plt.imshow(pil)
plt.figure()
plt.imshow(transformed_pil)


In [ ]:
import torchvision
from torchvision import transforms

def show_img(img):
  plt.figure(figsize=(40,38))
  npimg=img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))
  plt.show()

train_dataset.transform = transforms.Compose([                                                            
                              transforms.RandomRotation(50,expand=True),  
                              transforms.Resize((164,164)),
                              transforms.ToTensor(),
                              ])

Augmentation_dataloader=DataLoader(train_dataset,batch_size=8,shuffle=False)

data=iter(Augmentation_dataloader)
show_img(torchvision.utils.make_grid(data.next()[0]))

#### Создание собственных аугментаций

В том числе могут применяться в том числе  и к меткам!

In [ ]:
from PIL import ImageFilter

def blur(pil_image,radius):
  return pil_image.filter(ImageFilter.GaussianBlur(radius=radius))

class PadToSquare:
  def __call__(self, img):
    diff = pil_image.size[0] - pil_image.size[1]
    if diff == 0:
      return pil_image
    padding = (0,int(diff/2)) if diff >0 else (int(diff/2),0)
    return transforms.functional.pad(img,padding)

pil_image = Image.open("bike/bike_type/train/road/road_1150_5fccd8116e87e.jpeg")
transform=transforms.Compose([
                              transforms.Lambda(lambda x: blur(x,5)),
                              PadToSquare(),
                              ])
image = transform(pil_image)
plt.imshow(image)

#### Целесообразность

Не все аугментации имеют смысл для конкретной задачи.

Например VerticalFlip. A HorizontalFlip ?

#### Применение большого количества аугментаций может испортить изображение

In [ ]:
t_list = [                              
          transforms.RandomHorizontalFlip(),
          transforms.ColorJitter(brightness= (1,10), contrast=(1,10), saturation=(1,10), hue=0), # is chosen uniformly from ..
          transforms.CenterCrop(224),
          transforms.RandomRotation(50,expand=True),  
          transforms.Resize((164,164)),
          ]

t= transforms.Compose(t_list)

pil_image = Image.open("bike/bike_type/train/road/road_1150_5fccd8116e87e.jpeg")
transformed = t(pil_image)
plt.figure()
plt.imshow(pil_image)
plt.figure()
plt.imshow(transformed)

In [ ]:
# RandomChoice из torchvision.transforms 
# see also RandomOrder

plt.figure()
plt.imshow(pil_image)

for i in range(10):
  t = transforms.Compose([transforms.RandomChoice(t_list)])
  transformed = t(pil_image)
  plt.figure()
  plt.imshow(transformed)

### Online Augmentation

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/aug_online.png" width="700">




#### Аугментация как регуляризация


- помогает бороться с переобучением

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/aug_overfit.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/aug_reg.png" width="300">



#### Albumentation

Если стандартных аугментаций pytorch недостаточно, можно воспользоваться сторонними библиотеками.

Например:
https://github.com/albumentations-team/albumentations

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/album1.png" width="700">

Кроме расширенного набора аугментаций, albumentations позволяет применять их не только к самим изображениям, но к разметке: маскам, ограничивающим прямоугольникмам, ключевым точкам

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/album2.png" width="700">






In [ ]:
pip install -U albumentations

In [ ]:
import albumentations as A
import cv2

# Declare an augmentation pipeline
transform = A.Compose([
    A.RandomCrop(width=256, height=256),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.2),
])

transformed = t(pil_image)
plt.imshow(transformed)

### Unsupervised Learning

Если мы можем автоматически генерировать данные, то нельзя ли их и размечать без участия человека?

Можно всячески измениять данные, а потом учить сеть их восстанавливать:


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_16.png" width="700">


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/unsup2.png" width="700">

[Dosovitskiy et al., 2015](https://arxiv.org/abs/1406.6909)

[Gidaris et al. 2018](https://arxiv.org/abs/1803.07728)

[Doersch et al. (2015)](https://arxiv.org/abs/1505.05192)
[Noroozi, et al, 2017](https://arxiv.org/abs/1708.06734)

[Pathak, et al., 2016](https://arxiv.org/abs/1604.07379)



Зачем это нужно?

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/unsup1.png" width="700">

Веса которые мы получим в ходе такого обучения будут очень похожи на те которые сформировались бы при работе с размеченным человеком датасетом.
Особенно на первых слоях.









## Transfer learning

Для таких типовых задач, как классификация изображений, можно воспользоваться готовой архитектурой (AlexNet, VGG, Inception, ResNet и т.д.) и обучить нейросеть на своих данных. Реализации таких сетей с помощью различных фреймворков уже существуют, так что на данном этапе можно использовать одну из них как черный ящик, не вникая глубоко в принцип её работы.

Однако, глубокие нейронные сети требовательны к большим объемам данных для сходимости обучения. И зачастую, в нашей частной задаче недостаточно данных для того, чтобы хорошо натренировать все слои нейросети. `Transfer Learning` решает эту проблему. Зачем обучать сеть заново, если можно использовать уже обученную на миллионе изображений и дообучить на свой датасет?

В PyTorch есть много предобученных сетей: [TORCHVISION.MODELS](https://pytorch.org/vision/stable/models.html)

- AlexNet
- VGG
- ResNet
...

Для этого,  нужно отключить какие-то промежуточные слои. Тогда можно использовать то, что называется `Fine turning` - не нужно обучать всю модель, а достаточно только ее новую часть.

Зачастую в конце классификационных сетей используется полносвязный слой. Так как мы заменили этот слой, использовать предобученные веса для него уже не получится. Придется тренировать его с нуля, инициализировав его веса случайными значениями. Веса для всех остальных слоев мы загружаем из предобученной модели.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/L11_21.png" width="700">

Мы уже делали это когда сравнивали Resnet собственного изготовления библиотечной реализацией. Нам требовалось изменить колчество классов и мы просто заменяли у можели последний слой.

In [ ]:
from torchvision.models import resnet18

model = resnet18()
print(model)

Смотрим на вывод, линейный слой, и заменяем его другим с тем же количеством входов и нужным нам количеством выходов


In [ ]:
from torch import nn
model.fc = nn.Linear(model.fc.in_features, 10)

imagenet_input = torch.randn([1,3,224,224])

out = model(imagenet_input)
print(out)

## Заморозка весов




In [ ]:
for tag, param in model.named_parameters():
  if not 'layer4.' in tag and not 'fc' in tag:
  #if not any(map(tag.__contains__, ['layer4.','fc'])): another way
    param.requires_grad = False
  print(tag,type(param),param.shape,param.requires_grad)

In [ ]:
from torch import optim

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# or 

params_to_update = list(model.fc.parameters()) + list(model.layer4.parameters())
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

#### Замена и удаление произвольных слоев

При отправке тензора с высотой и шириной 32x32 возникает ошибка

In [ ]:
cifar10_input = torch.randn([1,3,32,32])
out = model(cifar10_input)
print(out)

Заменим 'stem' слои в начале сети, отвечающие за аггресивное сжатие изображения.

P.S. Это потребуется при выполнении практической работы.

In [ ]:
from torch import nn
model.conv1 = nn.Conv2d(3,64,kernel_size=(5, 5),stride = 1, padding =2, bias=False)
model.maxpool = nn.Identity()
out = model(cifar10_input)
print(out)


## Рекомендации по обучению


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/loss.png" width="700">

Обучение новой модели.
большой шаг ~1e-3

Дообучение (Transfer learning) 
маленикий шаг ~1e-5


*Выбор алгоритма оптимизации связан с выбором шагом обучения.



In [ ]:
Форматы изображений (OpenCv, Pillow ...)

### Сохранение весов

#Few shot learning

Не все задачи сводятся к классификации. Например распознавание лиц.





Предположим, мы хотим создать систему распознавания лиц для небольшой организации, в которой всего 10 сотрудников (небольшое количество упрощает задачу).

Используя традиционный подход к классификации, мы можем придумать систему, которая выглядит следующим образом:

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/OShL1.jpeg" width="700">

Проблемы:

а) Чтобы обучить такую ​​систему, нам сначала потребуется много разных изображений каждого из 10 человек в организации, что может оказаться невозможным. (Представьте, что вы делаете это для организации с тысячами сотрудников).

б) Что делать, если новый человек присоединяется к организации или покидает ее? Вам нужно снова взять на себя боль сбора данных и заново обучить всю модель.

Это практически невозможно, особенно для крупных организаций, где набор и увольнение происходит почти каждую неделю.

## Сиамские сети

Вместо того что бы классифицировать изображения можно использовать карту признаков (вектор) полученную на последнем слое и сравнивать эти вектора между собой.


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/siamese.png" width="800">



Можно кластеризовать эти вектора при помощи алгоритма кластеризации, и в зависимости от кого в какой кластер попал объект предсказывать его класс.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/few_shot.png" width="800">



### Косинусное расстояние

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/cos_dist.png" width="300">

Для сравнения эффективнее использовать косинусное расстояние. В отличие от Евклидова не зависит от масштаба.

In [ ]:
import torch.nn.functional as F
import torch
import numpy as np
import math

from  sklearn.metrics.pairwise import euclidean_distances

def dump_dist(a,b):
  euclidean_distance = F.pairwise_distance(a, b);
  cosine_distanse = F.cosine_similarity(a, b);

  print(f"Euclidean {euclidean_distance.item():.2f} ", )
  print(f"Cosine {cosine_distanse.item():.2f} deg. {math.degrees(math.acos(cosine_distanse)):.2f} ", )

a = torch.tensor([2.,5.]).view(1,-1) # Because all torch functions works with batches
b = torch.tensor([4.,2.]).unsqueeze(0) # the same
print(a,b)

dump_dist(a,b)

print("Now vectors are scaled")
a *= 10
b *= 10

dump_dist(a,b)


### Contrastive Loss

Для обучения потребуется специальная лосс-функция.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/contr_loss.png" width="800">

[2006 Dimensionality Reduction by Learning an Invariant Mapping](http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf)

Считается для набора значений. Где два принадлежат одному классу остальные нет

#### CosineEmbeddingLoss

https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html

В Pytorch реализован более удобный вариант требующий на вход пару изображений. Идея похожа на SVMLoss: дистанция между векторами из разных классов должна быть больше порога.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/cos_loss.png" width="800">

## Пример

In [ ]:
!wget http://edunet.kea.su/repo/src/L11_Transfer_learning/small_face_dataset.zip
!unzip small_face_dataset.zip

In [ ]:
#!wget http://www.anefian.com/research/GTdb_crop.zip
#!unzip GTdb_crop.zip
# http://conradsanderson.id.au/lfwcrop/ (LFWcrop Face Dataset, greyscale version)
#!wget http://conradsanderson.id.au/lfwcrop/lfwcrop_grey.zip
#!unzip lfwcrop_grey.zip
#!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz
#!tar zxvf lfw.tgz


In [ ]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision
import random
from PIL import Image
import torch
import numpy as np

class SiameseNetworkDataset(Dataset):
    
    def __init__(self,imageFolderDataset,transform=None):
        self.imageFolderDataset = imageFolderDataset    
        self.transform = transform        
    def __getitem__(self,index):
        img0_tuple = random.choice(self.imageFolderDataset.imgs)
        #we need to make sure approx 50% of images are in the same class
        should_get_same_class = random.randint(0,1) 
        if should_get_same_class:
            while True:
                #keep looping till the same class image is found
                img1_tuple = random.choice(self.imageFolderDataset.imgs) 
                if img0_tuple[1]==img1_tuple[1]:
                    break
        else:
            img1_tuple = random.choice(self.imageFolderDataset.imgs)

        img0 = Image.open(img0_tuple[0])
        img1 = Image.open(img1_tuple[0])
        img0 = img0.convert("L")
        img1 = img1.convert("L")
        


        if self.transform is not None:
            img0 = self.transform(img0)
            img1 = self.transform(img1)
        
  
        return img0, img1 ,int(img1_tuple[1] == img0_tuple[1])
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

folder_dataset = torchvision.datasets.ImageFolder(root="faces/training",transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor()
                                                                      ]))


siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor(),
                                                                      #transforms.Normalize(0.4371,0.1933)
                                                                      ]))




In [ ]:
import matplotlib.pyplot as plt
def imshow(img,text=None,should_save=False):
    npimg = img.numpy()
    plt.axis("off")
    if text:
        plt.text(75, 8, text, style='italic',fontweight='bold',
            bbox={'facecolor':'white', 'alpha':0.8, 'pad':10})
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show() 

vis_dataloader = DataLoader(siamese_dataset,
                        shuffle=True,
                        num_workers=2,
                        batch_size=8)
dataiter = iter(vis_dataloader)


example_batch = next(dataiter)
concatenated = torch.cat((example_batch[0],example_batch[1]),0)
plt.figure(figsize=(20,10))
imshow(torchvision.utils.make_grid(concatenated))
print('1 = самозванцы и 0 = совпадения : \n', example_batch[2].numpy())

In [ ]:
from torchvision.models import resnet18
from torch import nn
class SiameseNetwork(nn.Module):
  def __init__(self):
    super(SiameseNetwork, self).__init__()
    self.backbone = resnet18()
    self.backbone.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    self.backbone.fc = nn.Linear(self.backbone.fc.in_features, 128)

  def forward(self, input1, input2):
    output1 = self.backbone(input1)
    output2 = self.backbone(input2)
    return output1, output2



In [ ]:
import torch
from torch import nn, optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataloader = DataLoader(siamese_dataset,shuffle=True, num_workers=2, batch_size=64)
model = SiameseNetwork()
model.train()
model.to(device)
# https://pytorch.org/docs/stable/generated/torch.nn.CosineEmbeddingLoss.html
criterion = nn.CosineEmbeddingLoss(0.3)
optimizer = optim.Adam(model.parameters(),lr = 0.00005 )

counter = []
loss_history = [] 
iteration_number = 0

for epoch in range(30):
    for i, data in enumerate(train_dataloader,0):
        img0, img1 , label = data
        output1,output2 = model(img0.to(device),img1.to(device))
        optimizer.zero_grad()
        label[label == 0] = -1 # To capability with torch.nn.CosineEmbeddingLoss 
        loss = criterion(output1,output2,label.to(device))
        loss.backward()
        optimizer.step()
        if i %10 == 0 :
            print("Epoch number {} Current loss {}".format(epoch, loss.item()))
            iteration_number +=10
            counter.append(iteration_number)
            loss_history.append(loss.item())
            

In [ ]:
def show_plot(iteration,loss):
    plt.plot(iteration,loss)
    plt.grid()
    plt.show()
show_plot(counter[1:], loss_history[1:])

#### Test
Последние 3 фотографии были исключены из обучения и будут использоваться для тестирования. Расстояние между каждой парой изображений обозначает степень сходства модели между двумя изображениями.

Меньшее значение ошибки означает, что считает изображения похожими, в то время как более высокие значения указывают на то, что модель считает их разными людьми.


In [ ]:
import torch.nn.functional as F
folder_dataset_test = torchvision.datasets.ImageFolder(root='faces/testing')
siamese_dataset = SiameseNetworkDataset(imageFolderDataset=folder_dataset_test,
                                        transform=transforms.Compose([transforms.Resize((100,100)),
                                                                      transforms.ToTensor(),
                                                                      transforms.Normalize(0.4371,0.1933)
                                                                      ])
                                       )

test_dataloader = DataLoader(siamese_dataset,num_workers=2,batch_size=1,shuffle=True)
dataiter = iter(test_dataloader)
x0,_,_ = next(dataiter)

for i in range(10):
    x0,x1,label2 = next(dataiter)
    concatenated = torch.cat((x0,x1),0)
    
    output1,output2 = model(x0.to(device),x1.to(device))
    euclidean_distance = F.pairwise_distance(output1, output2).cpu()[0].item()
    cos_distance = F.cosine_similarity(output1, output2)[0].item();
    imshow(torchvision.utils.make_grid(concatenated),
           f'Dissimilarity: {euclidean_distance :.2f} {cos_distance :.2f}')

Учится но плохо...

## Предобработка

Для улучшения качества фотографии предобрабатывают

### Выравнивание

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/face_landmark.png" width="800">

Цель состоит в том что бы важные точки такие как глаза, рот и, оказались на одних и тех же местах. 

Такое преобразование называется [Гомография](https://waksoft.susu.ru/2020/03/26/primery-gomogrfii-s-ispolzovaniem-opencv/)






### Ключевые точки

Для получения матрицы гомографии  требуется знать какие точки в какие перейдут.

Для этого требуется знать где какие точки будут на изображении. И это отдельная задача компьютерного зрения (Landmark Detection).


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/face_pipeline.png" width="800">

### MTCNN

Раньше за процессы предобработки должен был отвечать программист. В 2016 году вышла статья описывающая моддель которая осуществляет всю предобработку.

[Joint Face Detection and Alignment using Multi-task Cascaded Convolutional Networks](https://arxiv.org/abs/1604.02878)







In [ ]:

!wget http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/peoples.jpg

## Triplet Loss

Другим усовершенствованием являетс замена лосс функции.


Триплет состоит из анкора `anchor`, положительного и отрицательного образцов и в основном применяется для распознавания лиц.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/OShL8.png" width="600">


В `Triplet loss` расстояние между базовым (якорным) изображением  и другим изображением лица того же человека минимизируется, а расстояние до негативного примера максимизируется.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/OShL9.png" width="600">

Функция расстояния может быть произвольной.

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/OShL10.png" width="600">




<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/triplet_loss.png" width="600">



https://pytorch.org/docs/stable/generated/torch.nn.TripletMarginLoss.htm


In [ ]:
import torch 
triplet_loss = torch.nn.TripletMarginLoss(margin=1.0, p=2)
anchor = torch.randn(100, 128, requires_grad=True)
positive = torch.randn(100, 128, requires_grad=True)
negative = torch.randn(100, 128, requires_grad=True)
loss = triplet_loss(anchor, positive, negative)
print(loss)
#output.backward()

Как видно для ее использования нужен датасет который возвращает три изображения.

## Метрики

Как использовать такого рода модели?

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/face_aim.png" width="800">


Выбирается порог.

И полученный на выходе модели вектор - признак сравнивается с эталонным или имеющимся в БД.






<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/metric.png" width="800">

От выбора порога зависит какие ошибки будет допускать модель.



Удобно использовать DET кривую - (detection error trade-off curve)
Что бы понять как количество ошибок первого рода влияет на количество второго.


<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/metric2.png" width="600">

### Кластеризация / Поиск

<img src ="http://edunet.kea.su/repo/src/L11_Transfer_learning/img/gan/face_clustering.png" width="600">

Фактически задача поиска решается методом K- ближайших соседей.

Для больших БД используются оптимизированные библиотеки:

[HNSW ](https://github.com/nmslib/nmslib)Efficient and robust approximate nearest neighbor search using Hierarchical Navigable Small World graphs
https://github.com/nmslib/nmslib


[Faiss](https://github.com/facebookresearch/faiss)
Faiss is a library for efficient similarity search and clustering of dense vectors.

### Демонстрация

Давате посмотрим как все это работает:
https://colab.research.google.com/drive/1OHgGRPc1sLcJK6_Q6o8vpUQ1G0IWgrHz